In [134]:
import sympy as sym
import numpy as np
import math
import scipy as sp

In [135]:
#Equation Setup
la,wa,ta,E,G,J,lb,wb,tb,lc,wc,tc,l,w,t,psi,d = sym.symbols('la,wa,ta,E,G,J,lb,wb,tb,lc,wc,tc,l,w,t,psi,d')

Ca = sym.Matrix([[0, 0, 0, la/(G*J), 0, 0],
             [0, 0, -6*la**2/(ta*wa**3*E), 0, 12*la/(ta*wa**3*E), 0],
             [0, 6*la**2/(ta**3*wa*E), 0, 0, 0, 12*la/(ta**3*wa*E)],
             [la/(ta*wa*E), 0, 0, 0, 0, 0],
             [0, 4*la**3/(ta**3*wa*E), 0, 0, 0, 6*la**2/(ta**3*wa*E)],
             [0, 0, 4*la**3/(ta*wa**3*E), 0, -6*la**2/(ta*wa**3*E), 0]])

Cb = sym.Matrix([[0, 0, 0, lb/(G*J), 0, 0],
             [0, 0, -6*lb**2/(wb*tb**3*E), 0, 12*lb/(wb*tb**3*E), 0],
             [0, 6*lb**2/(wb**3*tb*E), 0, 0, 0, 12*lb/(wb**3*tb*E)],
             [lb/(wb*tb*E), 0, 0, 0, 0, 0],
             [0, 4*lb**3/(wb**3*tb*E), 0, 0, 0, 6*lb**2/(wb**3*tb*E)],
             [0, 0, 4*lb**3/(wb*tb**3*E), 0, -6*lb**2/(wb*tb**3*E), 0]])

Cc = sym.Matrix([[0, 0, 0, lc/(G*J), 0, 0],
             [0, 0, -6*lc**2/(tc*wc**3*E), 0, 12*lc/(tc*wc**3*E), 0],
             [0, 6*lc**2/(tc**3*wc*E), 0, 0, 0, 12*lc/(tc**3*wc*E)],
             [lc/(tc*wc*E), 0, 0, 0, 0, 0],
             [0, 4*lc**3/(tc**3*wc*E), 0, 0, 0, 6*lc**2/(tc**3*wc*E)],
             [0, 0, 4*lc**3/(tc*wc**3*E), 0, -6*lc**2/(tc*wc**3*E), 0]])

R = sym.eye(3)

Da = sym.Matrix([[0,0,0],
             [0,0, lb+lc],
             [0,-lb-lc,0]])

Db = sym.Matrix([[0,0,0],
             [0,0,lc],
             [0,-lc,0]])

Ad_a = sym.Matrix([[R, sym.zeros(3)],
               [Da*R, R]])

Ad_b = sym.Matrix([[R, sym.zeros(3)],
               [Db*R, R]])

Ct = Ad_a*Ca*Ad_a.inv()+Ad_b*Cb*Ad_b.inv()+Cc
Ct_Assumed = sym.simplify(Ct.subs({ta:t,tb:t,tc:t,wa:w,wb:w,wc:w,la:l,lb:l,lc:l}))

R1 = sym.Matrix([[sym.cos((sym.pi+psi)/2), -sym.sin((sym.pi+psi)/2), 0],
             [sym.sin((sym.pi+psi)/2), sym.cos((sym.pi+psi)/2), 0],
             [0, 0, 1]])
#display(R1)

R2 = sym.Matrix([[sym.cos((sym.pi-psi)/2), -sym.sin((sym.pi-psi)/2), 0],
             [sym.sin((sym.pi-psi)/2), sym.cos((sym.pi-psi)/2), 0],
             [0, 0, 1]])
#display(R2)

D1 = sym.Matrix([[0, 0, 0],
             [0, 0, -d/2],
             [0, d/2, 0]])
#display(D1)

D2 = sym.Matrix([[0, 0, 0],
             [0, 0, d/2],
             [0, -d/2, 0]])
#display(D2)


Ad_1 = sym.Matrix([[R1, sym.zeros(3)],
               [D1*R1, R1]])
#display(Ad_1)

Ad_2 = sym.Matrix([[R2, sym.zeros(3)],
               [D2*R2, R2]])
#display(Ad_2)

Kt_Assumed = sym.cancel(Ct_Assumed.inv())
Kp_Assumed = sym.cancel(Ad_1*Kt_Assumed*Ad_1.inv() + Ad_2*Kt_Assumed*Ad_2.inv())

#Abbreviating Elements for Display
m, n = Kp_Assumed.shape
C = sym.Matrix(m, n, lambda i, j: sym.Symbol(f'k{i+1}{j+1}'))

# Loop through each element and replace non-zero entries with placeholders
for i in range(m):
    for j in range(n):
        if Kp_Assumed[i, j] == 0:
            C[i, j] = 0  # Retain sym.zeros

display(C)

Matrix([
[  0,   0, k13, k14,   0,   0],
[  0,   0,   0,   0, k25,   0],
[k31,   0,   0,   0,   0, k36],
[k41,   0,   0,   0,   0, k46],
[  0, k52,   0,   0,   0,   0],
[  0,   0, k63, k64,   0,   0]])

In [136]:
#Calculation
Load = sym.Matrix([150,-588.6,200,0,50,0])
t0,w0,l0,d0,psi0,E0,G0 = 1e-3,20e-3,20e-3,20e-3,math.radians(70),125e9,48e9
J0 = 0.312*w0*t0**3
Volume = t0*w0*l0*3*2 + t0*w0**2*4 + w0*d0**2
Mass = 4480*Volume
y_size = (3*l0+t0*4)*sym.cos(psi0/2)+d0/2
x_size = (3*l0+t0*3)*sym.sin(psi0/2)*2+d0
z_size = w0

Kp_Assumed_num = Kp_Assumed.subs({t:t0, w:w0, l:l0, d:d0, psi:psi0, E:E0, G:G0, J:J0})
Cp_Assumed_num = Kp_Assumed_num.inv()
freq_x = float(1/(2*sym.pi*sym.sqrt(Mass*Cp_Assumed_num[4-1,1-1])))
freq_z = float(1/(2*sym.pi*sym.sqrt(Mass*Cp_Assumed_num[6-1,3-1])))
Kp_Assumed_numpy = np.array(Kp_Assumed_num).astype(np.float64)
#eig_val = np.array(list(Kp_Assumed_num.eigenvals().keys()))
eig_val, eig_vec = np.linalg.eig(Kp_Assumed_numpy)
#display(Kp_Assumed_numpy)
#display(eig_vec)
#Kp_Assumed_num.eigenvects()[i][2][0] for i in range(3))
#display(type(eig_val))
#display(np.array(list(eig_val.keys())))
#display(1/(2*np.pi)*np.sqrt(abs(eig_val)))

Displacement = Kp_Assumed_num.solve(Load)
#Display Values
print(f"Mass = {Mass*1000:.2f} g")
print(f"X Size = {x_size*1000:.2f} mm\nY Size = {y_size*1000:.2f} mm\nZ Size = {z_size*1000:.2f} mm")
print(f"Natural Frequency in X Axis = {freq_x:.2f} Hz\nNatural Frequency in Z Axis = {freq_z:.2f} Hz")
print(f"X Displacement = {Displacement[3]*1000:.4f} mm\nY Displacement = {Displacement[4]*1000:.4f} mm\nZ Displacement = {Displacement[5]*1000:.4f} mm\nX Rotation = {math.degrees(Displacement[0]):.4f} deg\nY Rotation = {math.degrees(Displacement[1]):.4f} deg\nZ Rotation = {math.degrees(Displacement[2]):.4f} deg")

Mass = 53.76 g
X Size = 92.27 mm
Y Size = 62.43 mm
Z Size = 20.00 mm
Natural Frequency in X Axis = 384.57 Hz
Natural Frequency in Z Axis = 113.11 Hz
X Displacement = 0.4779 mm
Y Displacement = -0.0105 mm
Z Displacement = 7.3653 mm
X Rotation = 16.3776 deg
Y Rotation = 6.3956 deg
Z Rotation = 1.8998 deg


In [137]:
# Assuming you have already defined Kp_Assumed, E0, G0, etc.

# Objective function to minimize mass
def objective_minimize_mass(params, alpha = 0.2):
    w0, t0, l0, psi0, d0 = params
    # Calculate volume and mass
    Volume = t0 * w0 * l0 * 3 * 2 + t0 * w0**2 * 4 + w0 * d0**2
    Mass = 4480 * Volume
    
    # Calculate sizes
    x_size = (3 * l0 + t0 * 3) * math.sin(psi0 / 2) * 2 + d0
    y_size = (3 * l0 + t0 * 4) * math.cos(psi0 / 2) + d0 / 2
    z_size = w0
    
    # Combined objective to minimize mass and maximize sizes
    combined_value = Mass - alpha * (x_size + y_size + z_size)
    
    return combined_value

# Constraint functions to ensure sizes are maximized
def constraint_x_size(params):
    w0, t0, l0, psi0, d0 = params
    x_size = float((3 * l0 + t0 * 3) * math.sin(psi0 / 2) * 2 + d0)
    return float(x_size_max - x_size) # Ensure x size is below the maximum limit

def constraint_y_size(params):
    w0, t0, l0, psi0, d0 = params
    y_size = float((3 * l0 + t0 * 4) * math.cos(psi0 / 2) + d0 / 2)
    return float(y_size_max - y_size)  # Ensure y size is below the maximum limit

def constraint_z_size(params):
    w0, t0, l0, psi0, d0 = params
    z_size = float(w0)
    return float(z_size_max - z_size)  # Ensure z size is below the maximum limit

# Frequency constraints to ensure structural integrity
def constraint_freq_x(params):
    w0, t0, l0, psi0, d0 = params
    J0 = 0.312 * w0 * t0**3
    Kp_num = Kp_Assumed.subs({t: t0, w: w0, l: l0, d: d0, psi: psi0, E: E0, G: G0, J: J0})
    Cp_num = Kp_num.inv()
    Volume = t0 * w0 * l0 * 3 * 2 + t0 * w0**2 * 4 + w0 * d0**2
    Mass = 4480 * Volume
    freq_x = float(1 / (2 * math.pi * math.sqrt(Mass * Cp_num[4-1, 1-1])))
    return freq_x - 200  # Ensure natural frequency in x is greater than 200 Hz

def constraint_freq_z(params):
    w0, t0, l0, psi0, d0 = params
    J0 = 0.312 * w * t**3
    Kp_num = Kp_Assumed.subs({t: t0, w: w0, l: l0, d: d0, psi: psi0, E: E0, G: G0, J: J0})
    Cp_num = Kp_num.inv()
    Volume = t0 * w0 * l0 * 3 * 2 + t0 * w0**2 * 4 + w0 * d0**2
    Mass = 4480 * Volume
    freq_z = float(1 / (2 * math.pi * math.sqrt(Mass * Cp_num[6-1, 3-1])))
    return freq_z - 200  # Ensure natural frequency in z is greater than 200 Hz

# Define the maximum allowed sizes in x, y, z directions
x_size_max = 0.100  # Example: 100 mm
y_size_max = 0.065  # Example: 100 mm
z_size_max = 0.030  # Example: 100 mm

# Initial guesses
initial_guess = [0.02, 0.002, 0.05, math.radians(60), 0.02]  # Example initial guesses for w, t, l, psi, d0

# Bounds for parameters using scipy's Bounds class
bounds = sp.optimize.Bounds([0.001, 0.001, 0.01, math.radians(30), 0.01], [0.15, 0.15, 0.15, math.radians(120), 0.1], True)

# Constraints for minimize function
constraints = [
    {'type': 'ineq', 'fun': constraint_x_size},
    {'type': 'ineq', 'fun': constraint_y_size},
    {'type': 'ineq', 'fun': constraint_z_size},
    {'type': 'eq', 'fun': constraint_freq_x},
    {'type': 'eq', 'fun': constraint_freq_z}
]

# Perform Basin Hopping
minimizer_kwargs = {"method": "SLSQP", "bounds": bounds, "constraints": constraints}
result = sp.optimize.basinhopping(objective_minimize_mass, initial_guess, minimizer_kwargs=minimizer_kwargs, niter=5, disp = True)

# Function to calculate mass, natural frequencies, and sizes
def calculate_properties(params):
    w0, t0, l0, psi0, d0 = params
    # Calculate mass
    Volume = t0 * w0 * l0 * 3 * 2 + t0 * w0**2 * 4 + w0 * d0**2
    Mass = 4480 * Volume
    
    # Calculate natural frequencies
    J0 = 0.312 * w0 * t0**3
    Kp_num = Kp_Assumed.subs({t: t0, w: w0, l: l0, d: d0, psi: psi0, E: E0, G: G0, J: J0})
    Cp_num = Kp_num.inv()
    freq_x = float(1 / (2 * math.pi * math.sqrt(Mass * Cp_num[4-1, 1-1])))
    freq_z = float(1 / (2 * math.pi * math.sqrt(Mass * Cp_num[6-1, 3-1])))
    
    # Calculate sizes
    x_size = float((3 * l0 + t0 * 3) * math.sin(psi0 / 2) * 2 + d0)
    y_size = float((3 * l0 + t0 * 4) * math.cos(psi0 / 2) + d0 / 2)
    z_size = float(w0)
    
    return Mass, freq_x, freq_z, x_size, y_size, z_size

if result.success:
    print('Optimization is successful')
else:
    print('Optimization failed')
optimized_params = result.x
optimized_mass, optimized_freq_x, optimized_freq_z, optimized_x_size, optimized_y_size, optimized_z_size = calculate_properties(optimized_params)
print("\nOptimization Converged!")
print(f"Optimized Width (w): {optimized_params[0]*1000:.2f} mm")
print(f"Optimized Thickness (t): {optimized_params[1]*1000:.2f} mm")
print(f"Optimized Length (l): {optimized_params[2]*1000:.2f} mm")
print(f"Optimized Angle (psi): {math.degrees(optimized_params[3]):.2f} degrees")
print(f"Optimized Diameter (d0): {optimized_params[4]*1000:.2f} mm")
print(f"Optimized Mass: {optimized_mass:.2f} kg")
print(f"Natural Frequency in X Direction: {optimized_freq_x:.2f} Hz")
print(f"Natural Frequency in Z Direction: {optimized_freq_z:.2f} Hz")
print(f"Optimized Size X: {optimized_x_size*1000:.2f} mm")
print(f"Optimized Size Y: {optimized_y_size*1000:.2f} mm")
print(f"Optimized Size Z: {optimized_z_size*1000:.2f} mm")

basinhopping step 0: f 0.47064
basinhopping step 1: f 0.47064 trial_f 6.06315 accepted 0  lowest_f 0.47064
basinhopping step 2: f 0.47064 trial_f 7.5334 accepted 0  lowest_f 0.47064
basinhopping step 3: f 0.492333 trial_f 0.492333 accepted 1  lowest_f 0.47064
basinhopping step 4: f 0.511363 trial_f 0.511363 accepted 1  lowest_f 0.47064
basinhopping step 5: f 0.49393 trial_f 0.49393 accepted 1  lowest_f 0.47064
Optimization failed

Optimization Converged!
Optimized Width (w): 30.00 mm
Optimized Thickness (t): 1.73 mm
Optimized Length (l): 10.00 mm
Optimized Angle (psi): 30.00 degrees
Optimized Diameter (d0): 58.71 mm
Optimized Mass: 0.51 kg
Natural Frequency in X Direction: 320.37 Hz
Natural Frequency in Z Direction: 265.91 Hz
Optimized Size X: 76.92 mm
Optimized Size Y: 65.00 mm
Optimized Size Z: 30.00 mm
